<a href="https://colab.research.google.com/github/abenben/DXF2020/blob/main/DXF2020_1113.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Agenda
- 次元削減とクラスタリング（教師なし学習）
- 回帰（教師あり、目的変数が連続値）
- 分類（教師あり、目的変数が離散値）

In [ ]:
# 可視化のための外部モジュールの読み込み
import matplotlib.pyplot as plt
# ノートブックの中に画を埋め込むための指示
%matplotlib inline

# データサイエンスによく使うライブラリも読み込んでおく
import numpy as np
import pandas as pd

In [ ]:
# 手書き数字サンプルデータの読み込み準備
from sklearn.datasets import load_digits

In [ ]:
# 関数を呼び出してサンプルデータを読み込み変数（digits_data）で受け取る
# 詳しくは、以下の公式ドキュメントを参照
# https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits
digits_data = load_digits()

In [ ]:
# 簡単な説明書の表示
# （実は間違っている。1797個しがデータが無い）
print(digits_data.DESCR)

In [ ]:
#試しに1つ表示して見る
# 表示するデータのインデックスを指定
i = 10 # 0〜1798までで指定できる
# 変数で受け取る
image = digits_data['images'][i]
num = digits_data['target'][i]
print(f'ラベルは{num}')
# 画像の表示
_ = plt.imshow(image, cmap=plt.cm.binary) #またはgray_rでもOK

# 試してみよう
# iの値を変更してみる
# カラーマップを変更してみる
# https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html

In [ ]:
# 1つの画像は8行8列の行列データ
digits_data['images'][i]

In [ ]:
# ベクトルになっているデータもある
digits_data['data'][i]

In [ ]:
# 全体をまとめて1つの表型のデータを作る
# 通常は行と列に名前を付ける
digits_df = pd.DataFrame(digits_data['data'])

In [ ]:
# 1797行、64列のデータ
# 1行が1つのサンプル。それぞれの列が説明変数
digits_df

In [ ]:
# サンプルを2次元平面にプロットするための便利関数を作る
def plot_2D(X, y, file_name=None):
    plt.figure()
    ax = plt.subplot(111)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='jet')
    plt.colorbar()
    #ax.set_aspect('equal')
    if file_name:
        plt.savefig(file_name)
        plt.close()

# しかし、64次元のデータをどうやって、2次元へ・・・？

In [ ]:
# ここで使われるのが次元削減の手法
# まずはもっとも古典的なPCA（主成分分析）から
from sklearn.decomposition import PCA
# 2次元データを出力するPCAのインスタンスを用意
pca = PCA(n_components=2)
# digits_data['data']でもOK
digits_pca = pca.fit_transform(digits_df)
# digits_data['target']に正解（0〜9までの数字）が入っているので色が付く
plot_2D(digits_pca, digits_data['target'])

In [ ]:
# 次元削減にはいろいろな方法がある
# 最近は、ｔ−SNE（t-distributed Stochastic Neighbor Embedding）がよく使われる
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
digits_tsne = tsne.fit_transform(digits_df)
plot_2D(digits_tsne, digits_data['target'])

次元削減を実行するときのパラメータが手法ごとにいろいろある

perplexityfloat, optional (default: 30)

The perplexity is related to the number of nearest neighbors that is used in other manifold learning algorithms. Larger datasets usually require a larger perplexity. Consider selecting a value between 5 and 50. Different values can result in significanlty different results.

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, perplexity=10)
digits_tsne = tsne.fit_transform(digits_df)
plot_2D(digits_tsne, digits_data['target'])

# 試してみよう
# perplexityを適当に設定してみよう
# 同じパラメータ設定でも違う結果がでます。どうしたら固定できるでしょうか？

In [ ]:
from sklearn.manifold import TSNE
# random_stateを固定することで、結果を再現できる
# 詳しくは、t-SNEの中身を知る必要がある
tsne = TSNE(n_components=2, perplexity=30, random_state=0)
digits_tsne = tsne.fit_transform(digits_df)
plot_2D(digits_tsne, digits_data['target'])

In [ ]:
# scikit-learnのサイトからK-meansクラスタリングのイメージ
from IPython.core.display import Image, display
display(Image("https://scikit-learn.org/stable/_images/sphx_glr_plot_kmeans_digits_001.png"))

In [ ]:
from sklearn.cluster import KMeans
# 入力データを10クラスに分ける
kmeans = KMeans(n_clusters=10, random_state=0)
kmeans.fit(digits_df)
# サンプルが属するクラスター
kmeans.labels_

In [ ]:
from sklearn.metrics import silhouette_score

for n_cluster in range(5, 16):
    clusterer = KMeans(n_clusters=n_cluster, random_state=10)
    cluster_labels = clusterer.fit_predict(digits_df)
    silhouette_avg = silhouette_score(digits_df, cluster_labels)
    print("For n_clusters =", n_cluster,
          "The average silhouette_score is :", silhouette_avg)

# 詳しくは
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py

In [ ]:
# パラメータにクラス数を指定しなくてもよいクラスタリング手法もある
from sklearn.cluster import AffinityPropagation
# random_stateの設定は、バージョン0．23から
clustering = AffinityPropagation().fit(digits_df)
# 何クラスに分かれたか？
len(set(clustering.labels_))

In [ ]:
import seaborn as sns
sns.clustermap(digits_df)

In [ ]:
# 説明変数の分散のヒストグラム
plt.hist(digits_df.std())

In [ ]:
# 前処理の一例
# 分散が小さい（サンプル間でほとんどばらつきがない）説明変数を削除
idx = digits_df.std() > 2
filtered = digits_df[digits_df.columns[idx]]

sns.clustermap(filtered)

# 試してみよう
# 分散での前処理を調整して、いくつか階層的クラスタリングの図を描いてみよう

In [ ]:
from sklearn.datasets import load_boston
boston_data = load_boston()

In [ ]:
print(boston_data.DESCR)

In [ ]:
# 住宅の価格（＄1，000）
y = boston_data['target']
# 説明変数を準備
X = boston_data['data']

In [ ]:
# DataFrameを作ります。
boston_df = pd.DataFrame(boston_data.data)
# 列名をつけます。
boston_df.columns = boston_data.feature_names
# 便利のために、価格列を追加します。
boston_df['PRICE'] = y
boston_df.head()

In [ ]:
# 横軸に部屋数、縦軸に価格
boston_df.plot.scatter('RM', 'PRICE')

# 試してみよう
# X軸を変更してみてください。以下が比較的意味をとりやすいかも。
# 犯罪率　CRIM
# 窒素酸化物濃度 NOX
# 生徒と先生の費　PTRATIO

In [ ]:
# seabornを使うと簡単に回帰直線を描けます
sns.lmplot('RM', 'PRICE', data = boston_df)

In [ ]:
# 便宜的に、訓練データと、目的変数を知らないことにするテストデータに分ける
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# 全部で506サンプルあったが、訓練用とテスト用に分けられた。
print(X_train.shape, X_test.shape)

In [ ]:
# 線形重回帰を使ったモデルを作る
from sklearn.linear_model import LinearRegression
# インスタンスを作って、訓練データからモデルを作成
reg = LinearRegression().fit(X_train, y_train)

In [ ]:
# 線形重回帰なので、各変数の係数がわかる
print(reg.coef_)
print(reg.intercept_)

In [ ]:
#　予測と当てはまりの良さを計算
from sklearn.metrics import mean_squared_error, r2_score

# 未知のサンプルの価格を予測
y_pred = reg.predict(X_test)
# The mean squared error
print('平均2乗誤差: ', mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('決定係数: ', r2_score(y_test, y_pred))

In [ ]:
# 正解データと予測結果を図示
plt.scatter(y_test, y_pred)
plt.xlabel('test')
plt.ylabel('pred')

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(n_estimators=10, max_depth=3, random_state=0)
rf_reg.fit(X_train, y_train)

In [ ]:
rf_reg_pred = rf_reg.predict(X_test)

print('平均2乗誤差: ', mean_squared_error(y_test, rf_reg_pred))
print('決定係数: ', r2_score(y_test, rf_reg_pred))

# 試してみよう
# n_estimatorsやmax_depthを変更するとどうなるでしょう？

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'n_estimators':[50, 100, 200], 'max_depth': [2, 4, 8, 16]}
rf_reg = RandomForestRegressor()
clf = GridSearchCV(rf_reg, parameters)
_ = clf.fit(X_train, y_train)

In [ ]:
# 最も性能がよいモデルで予測
rf_reg_pred = clf.best_estimator_.predict(X_test)

print('平均2乗誤差: ', mean_squared_error(y_test, rf_reg_pred))
print('決定係数: ', r2_score(y_test, rf_reg_pred))

In [ ]:
# 使われたパラメータを表示
clf.best_estimator_.get_params

In [ ]:
# 勾配ブースティングという方法が性能が良いので最近はよく使われている。
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor().fit(X_train, y_train)

gbr_pred = gbr.predict(X_test)
print('平均2乗誤差: ', mean_squared_error(y_test, gbr_pred))
print('決定係数: ', r2_score(y_test, gbr_pred))

In [ ]:
# ワインサンプルデータの準備
from sklearn.datasets import load_wine
wine_data = load_wine()

In [ ]:
# データの説明
print(wine_data.DESCR)

In [ ]:
#　分類わけは数字で入っている
wine_data['target']

In [ ]:
# PCAで2次元に落としこむ
pca = PCA(n_components=2)
wine_pca = pca.fit_transform(wine_data['data'])
plot_2D(wine_pca, wine_data['target'])

In [ ]:
# データの正規化（規格化）
# 変数ごとに平均を引いて標準偏差で割るという処理をする。
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss_data = ss.fit_transform(wine_data['data'])
# PCAで2次元へ
pca = PCA(n_components=2)
ss_pca = pca.fit_transform(ss_data)
plot_2D(ss_pca, wine_data['target'])

In [ ]:
# 2クラスの分類とROCによる評価

# SVM(support vector machine)を準備
from sklearn.svm import SVC
# ついでにRandom Forestsも準備
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_roc_curve

X = wine_data['data']
# 0と1を0に、2を1に変換
y = wine_data['target'].copy()
y[y == 1] = 0
y[y == 2] = 1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
svc = SVC(random_state=42)
svc.fit(X_train, y_train)

In [ ]:
# 作ったモデルで予測
svc_pred = svc.predict(X_test)

In [ ]:
# 全部0になってしまっている
svc_pred

In [ ]:
# ほんとは1のサンプルもある
y_test

In [ ]:
# より詳しい結果を計算できる
from sklearn.metrics import classification_report

print(classification_report(y_test , svc_pred))

In [ ]:
# サンプルごとに予測の自信度は異なる
svc.decision_function(X_test)

In [ ]:
# 受信者操作特性 Receiver operating characteristic 
svc_disp = plot_roc_curve(svc, X_test, y_test)
plt.show()

In [ ]:
# RandomForestsを使ってみる
# 直前に作ったSVMのROCに追加するためのコードが入っているので、ちょっとわかりにくい
rfc = RandomForestClassifier(n_estimators=10, random_state=42)
rfc.fit(X_train, y_train)
ax = plt.gca()
rfc_disp = plot_roc_curve(rfc, X_test, y_test, ax=ax, alpha=0.8)
svc_disp.plot(ax=ax, alpha=0.8)
plt.show()

In [ ]:
# もともとの3クラスの分類
# もう一度ｙを代入しなおす
y = wine_data['target'].copy()
# 訓練用とテスト用に分ける
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# SVMは2クラス用だが、マルチクラスにもそのまま使える
svc = SVC(random_state=42)
svc.fit(X_train, y_train)
svc_pred = svc.predict(X_test)
svc_pred

In [ ]:
print(classification_report(y_test , svc_pred))

In [ ]:
# 混合行列を使った可視化
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, svc_pred)
conf_mat

In [ ]:
# 行方向が正解、列方向が予測
#　クラス1のrecall（再現率）は、
print(13/(13+5))
# クラス1のpredision（適合率）は
print(13/(13+8))
# 全体のaccuracy（正解率）は、
print((15+13+4)/45)

In [ ]:
# ヒートマップを使った可視化がわかりやすい
sns.heatmap(conf_mat, annot=True)

In [ ]:
# RandomForestsがいい
rfc = RandomForestClassifier(n_estimators=10, random_state=42)
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)
print(classification_report(y_test , rfc_pred))

In [ ]:
# どの変数が分類に効いているかがわかる。
fi = rfc.feature_importances_
# ただの棒グラフを描くのがちょっと面倒だったりする・・・。
plt.bar(range(len(fi)), fi, tick_label=wine_data['feature_names'])
plt.xticks(rotation=90)

# やってみよう
# 1つ前のセルで作ったRandomForestsのインスタンスで、random_stateの数字を変えると、feature importanceがどうなるか試してみてください。

In [ ]:
# 卒業試験
# 以下のGradientBoostingClassifierを使って、予測モデルを作り、その精度を計算してください。feature importanceの棒グラフも描いてください。
from sklearn.ensemble import GradientBoostingClassifier

## Appendix

[決定係数の説明(Wikipedia)](https://ja.wikipedia.org/wiki/%E6%B1%BA%E5%AE%9A%E4%BF%82%E6%95%B0)

[ROCの説明(Wikipedia)](https://ja.wikipedia.org/wiki/%E5%8F%97%E4%BF%A1%E8%80%85%E6%93%8D%E4%BD%9C%E7%89%B9%E6%80%A7)

[決定木、RandomForests、勾配ブースティングのわかりやすいページ](https://www.codexa.net/lightgbm-beginner/)